# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 6c337025


In [8]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [9]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [10]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [11]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [14]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/karlaudiljak/AIE9/07_Deep_Agents/workspace


In [15]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
(empty directory)


In [16]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [17]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [18]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/karlaudiljak/AIE9/07_Deep_Agents/workspace


In [20]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
### **Your Next Steps:**

1. **Read the full plan:** Review `/comprehensive_sleep_improvement_plan.md` 
2. **Start Week 1 tonight:** Pick your bedtime and remove your phone from the bedroom
3. **Track your progress:** Note how you feel each morning
4. **Be patient:** Quality sleep improvements typically show results in 2-4 weeks

The plan is designed to be realistic and sustainable - you don't have to change everything at once. Focus on consistency over perfection, and remember that even small improvements in sleep can have significant impacts on your energy, mood, and overall health.

Would you like me to clarify any part of the plan or help you customize any specific recommendations?


In [21]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research sleep hygiene best practices (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_9] Provide implementation guidance (completed)
✅ [todo_6] Research sleep schedule consistency and circadian rhythm optimization (completed)
✅ [todo_8] Research digital device/blue light management strategies (completed)
✅ [todo_10] Research evening routines and wind-down activities (completed)
✅ [todo_12] Research sleep environment optimization (completed)
✅ [todo_14] Research morning routines for better wake-up energy (completed)
✅ [todo_16] Research lifestyle factors affecting sleep (completed)
✅ [todo_18] Develop gradual implementation strategies (completed)
✅ [todo_20] Address specific issues: inconsistent bedtime, phone use, morning fatigue (completed)
✅ [todo_22] Compile comprehensive sleep improvement plan document (completed)


Work

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
*Todo lists are great for long, messy tasks, but for small, straightforward ones they can just slow things down with extra planning steps, where acting directly would probably be faster. Todo items shouldn’t be too big, or the agent can’t clearly tell what’s done, but they also shouldn’t be too tiny, like tiny micro-steps, because then the agent wastes time managing them and can get stuck in loops. So, todo items should be granular enough to be actionable, but not so detailed that they turn into busywork or cause loops. If an agent makes todos but never finishes them, they just sit there and get in the way, making it look like progress is being made when it’s not. This can also make the agent keep redoing or rethinking the same things. Todo lists work best when the agent only makes them when needed and keeps them organized and up to date.*

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
*I’d keep the big health document in files or a retrieval system and only pull in the small parts that matter for the current question, instead of dumping the whole 16KB into the prompt. User metrics over time, like sleep or mood, should live in structured storage and be summarized into simple trends when needed. The prompt itself should stay focused on the current goal and a quick snapshot of what’s going on right now. What should never be offloaded are safety-critical details like allergies, medications, medical conditions, and past bad reactions. Those need to be immediately available so the agent doesn’t make unsafe suggestions. The current plan and key constraints should also stay in context to avoid confusion.*

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [52]:
### YOUR CODE HERE ###
# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide

# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")
@tool
def read_health_wellness_guide():
    """Read the HealthWellnessGuide.txt file from the data folder."""
    return Path("data/HealthWellnessGuide.txt").read_text()

# Step 3: Create the agent with a research-focused system prompt
research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos,
    update_todo,
    list_todos, read_health_wellness_guide], 
    backend=filesystem_backend,  
    system_prompt="""
    You are a Wellness Research Agent.

When given a task:
1. Create todos for each major step in the research process.
2. Read from the HealthWellnessGuide.txt in the data folder.
3. Create a folder called reports if it doesn't exist.
4. Produce a structured report based on findings and save to a structured Markdown file in the reports folder.
5. Update todo status after each step.

The Markdown report must include:
- Title
- Executive Summary
- Key Topics (with bullet points)
- Practical Recommendations
- References (always including sections from the HealthWellnessGuide.txt)

CRITICAL:
Before finishing, call list_todos.
If any todo is not marked "completed", update it immediately.
Do not end execution until ALL todos have status "completed".

Be thorough but concise. Always keep todos up to date."""
)


In [53]:
# Step 4: Test with the stress management research task
TODO_STORE.clear()

# Test with a multi-step wellness task
result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Excellent! All todos are now completed. 

## Summary

I have successfully created a **Comprehensive Stress Management Guide** with 12 evidence-based strategies that goes far beyond the basic 5 requested. The guide has been saved to `/reports/comprehensive_stress_management_guide.md` and includes:

### **12 Evidence-Based Stress Management Strategies:**

**Foundational Techniques (3):**
- Deep Breathing and Box Breathing
- Progressive Muscle Relaxation 
- 5-4-3-2-1 Grounding Technique

**Intermediate Approaches (3):**
- Basic Mindfulness Meditation
- Regular Physical Exercise
- Social Connection and Support

**Advanced Interventions (6):**
- Cognitive Restructuring (CBT Approach)
- Advanced Mindfulness (Body Scan & Loving-Kindness)
- Cold Exposure Therapy
- Expressive Writing (Structured Journaling)
- Heart Rate Variability (HRV) Biofeedback
- Nature Immersion and Forest Bathing

### **Key Features of the Guide:**

✅ **Evidence-Based**: Each technique includes specific r

In [54]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

Todo list after task:
✅ [todo_1] Read HealthWellnessGuide.txt (completed)
✅ [todo_3] Create reports folder (completed)
✅ [todo_5] Research evidence-based stress management techniques (completed)
✅ [todo_7] Analyze existing stress management content (completed)
✅ [todo_9] Save report to reports folder (completed)
✅ [todo_11] Verify all todos completed (completed)
✅ [todo_8] Research cognitive-behavioral techniques (completed)
✅ [todo_10] Research advanced mindfulness and meditation practices (completed)
✅ [todo_12] Research physiological interventions (completed)
✅ [todo_14] Research creative and expressive therapies (completed)
✅ [todo_16] Research social and environmental modifications (completed)
✅ [todo_18] Research technology-assisted techniques (completed)
✅ [todo_20] Compile comprehensive report (completed)


In [55]:
print("\nGenerated files in workspace:")
reports_dir = WORKSPACE / "reports"
for f in sorted(reports_dir.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")


Generated files in workspace:
  [FILE] .gitkeep (46 bytes)
  [FILE] comprehensive_stress_management_guide.md (10156 bytes)


---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [35]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [42]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [43]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I've delivered:

## ✅ **Complete Project Summary:**

**📚 Research Foundation**
- Comprehensive scientific research on circadian rhythms, exercise physiology, nutrition timing, sleep science, psychological factors, and hormone regulation
- All recommendations are evidence-based and backed by current research

**📖 Comprehensive Guide Features**
- **Science-based framework**: WAKE-MOVE-FUEL-FOCUS-FLOW system
- **Multiple routine options**: 15-minute, 30-minute, and 60-minute versions
- **Practical adaptations**: For parents, travelers, shift workers, office workers, and remote workers
- **Implementation strategy**: Week-by-week progression plan
- **Troubleshooting**: Common pitfalls and solutions
- **Customization options**: For different energy levels, fitness levels, seasons, and goals

**💾 File Saved**
The complete guide has been saved as `/morning_routine_en

In [44]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines and energy (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save the final guide as a markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive-morning-routine-guide.md (14559 bytes)
  [FILE] comprehensive_sleep_improvement_plan.md (27148 bytes)
  [FILE] morning_routine_energy_guide.md (14919 bytes)
  [FILE] morning_routine_guide.md (3331 bytes)
  [DIR] reports/
  [DIR] research/


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [56]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [57]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [58]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [59]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! It's great to hear from you again. Based on your profile, I remember that your main goals are to improve your energy levels and get better sleep, and that you prefer morning workouts. I also note that you have mild anxiety, which I'll keep in mind for my recommendations.

Here's a personalized exercise routine that should help boost your energy and improve your sleep quality:

## **Morning Energy-Boosting Routine (30-40 minutes)**

### **Monday, Wednesday, Friday - Cardio & Strength**
- **5-minute warm-up**: Light stretching or gentle yoga poses
- **20 minutes cardio**: Brisk walking, cycling, or swimming (anxiety-friendly, low-impact options)
- **10-15 minutes strength training**: Bodyweight exercises like push-ups, squats, lunges, and planks
- **5-minute cool-down**: Deep breathing exercises and gentle stretching

### **Tuesday, Thursday - Yoga & Mindfulness**
- **25-30 minutes**: Morning yoga flow focusing on energizing poses (Sun Salutations, Warrior poses)

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [60]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [61]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [62]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [63]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [64]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Wellness Assessment

Based on your information, I've identified **sleep optimization** and **movement integration** as your top priorities, as these will have the biggest impact on your 15-pound weight loss goal.

### 🎯 **Immediate Action Items** (Start Today)
1. **Set a sleep schedule**: Choose a consistent bedtime and wake time, aiming for 7-8 hours
2. **Add hourly movement**: Set phone reminders to stand/stretch every hour during work
3. **Track your baseline**: Note your current sleep hours and daily steps for 3 days

### 📈 **Short-term Goals** (1-2 weeks)
1. **Establish movement breaks**: 5-minute walks or stretches every 2 hours during work
2. **Create a wind-down routine**: 30 minutes of relaxing activities before bed (no screens)
3. **Plan protein-rich vegetarian meals**: Focus on legumes, quinoa, Greek yogurt, nuts

### 🏆 **Long-term Goals** (1-3 months)
1. **Regular exercise routine**: 150 minutes moderate activity per week + 2 strength se

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [65]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.17


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [66]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [67]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [68]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
## Your 2-Week Wellness Program is Complete! 🎉

I've successfully created a comprehensive, personalized wellness program tailored specifically to your needs and preferences. Here's what you now have:

### **Your Complete Program Files**:

1. **`/alex_2week_exercise_plan.md`** - Your complete workout guide
   - 3x per week, 30-minute morning workouts
   - Progressive 2-week structure with anxiety-friendly elements
   - Specific exercises, modifications, and mindfulness components

2. **`/alex_2week_nutrition_plan.md`** - Your vegetarian meal planning system
   - 14-day complete meal plan with energy-boosting foods
   - Shopping lists, meal prep tips, and sleep-supporting evening meals
   - All vegetarian with focus on anxiety management

3. **`/alex_2week_stress_sleep_plan.md`** - Your stress & sleep optimization guide
   - Daily stress management routines
   - Comprehensive evening wind-down protocols
   - Workplace stress strategies and anxiety-friendly techni

In [69]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create 2-week exercise routine plan (completed)
✅ [todo_3] Develop vegetarian meal plan (completed)
✅ [todo_5] Design stress management and sleep optimization plan (completed)
✅ [todo_7] Integrate all plans into comprehensive program (completed)
✅ [todo_9] Save exercise plan to file (completed)
✅ [todo_11] Save nutrition plan to file (completed)
✅ [todo_13] Save stress/sleep plan to file (completed)
✅ [todo_15] Create master wellness program file (completed)

GENERATED FILES
  [FILE] advanced_stress_management_report.md (17554 bytes)
  [FILE] alex_2week_exercise_plan.md (4984 bytes)
  [FILE] alex_2week_nutrition_plan.md (7537 bytes)
  [FILE] alex_2week_stress_sleep_plan.md (9940 bytes)
  [FILE] alex_comprehensive_2week_wellness_program.md (9012 bytes)
  [FILE] comprehensive-morning-routine-guide.md (14559 bytes)
  [FILE] comprehensive_sleep_improvement_plan.md (27148 bytes)
  [FILE] comprehensive_stress_management_research.md (36644 bytes)
  [FILE] exercise

In [70]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of alex_comprehensive_2week_wellness_program.md:
# Alex's Complete 2-Week Wellness Transformation Program

## Program Overview
Welcome to your personalized wellness journey! This comprehensive program integrates exercise, nutrition, and stress management to help you achieve your goals of building consistent habits, improving energy levels, and getting better sleep.

## Your Personalized Program Components

### 🏃‍♂️ **Exercise Component** (3x/week, 30 minutes)
**File**: `alex_2week_exercise_plan.md`
- Morning workouts designed for energy boost
- Anxiety-friendly movements with mindfulness elements
- Progressive 2-week structure building from foundation to confidence
- Includes warm-ups, main workouts, and calming cool-downs

### 🥗 **Nutrition Component** (Vegetarian focus)
**File**: `alex_2week_nutrition_plan.md`
- Complete 14-day vegetarian meal plan
- Energy-boosting food combinations
- Sleep-supporting evening meals
- Shopping lists and meal prep guides
- Special attention 

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
*I would share tools when they’re read-only or low risk, and give subagents separate tools when there are side effects or security concerns.
I would use stronger models for planning and synthesis, and cheaper models for extraction, formatting, or validation.
I’d match model cost to task complexity and avoid using expensive models for simple tasks.
I would keep subagent granularity at the role level, not the micro-task level, meaning each agent owns a complete responsibility like planning, research, writing, or execution.
I wouldn’t make agents for tiny things like “write bullets” or “format paragraphs” because that creates lots of coordination overhead and makes the system fragile. I would aim for agents that can work independently for a meaningful chunk of the workflow.
Too few agents make the system harder to manage, while too many agents slow everything down.
Overall, I’d go with one main “brain” and a few focused helpers rather than lots of tiny agents.*

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
*I’d add safety guardrails to ensure health advice stays within evidence-based, non-diagnostic boundaries (plus escalation paths to human review for anything sensitive).
I’d replace in-memory state with persistent storage and introduce per-user isolation so each user’s data, sessions, and agent state are fully separated.
I’d layer in some kind of authentication/authorization and tenant-aware routing to support multiple users safely.
I’d implement monitoring and observability (like logs, traces, metrics, prompt/tool usage, failure alerts) so that I can debug agent behavior in real time.
Finally, I’d control costs by using cheaper models when possible, limiting how much each user can spend, reusing previous results, and running some work in the background, so expenses stay predictable.*

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [74]:
### YOUR CODE HERE ###
from langgraph.checkpoint.memory import MemorySaver
checkpointer = MemorySaver()
# Step 1: Define your subagent configurations
fitness_tracker = {
    "name": "fitness-tracker",
    "description": "Tracks workouts, physical activity, and fitness progress over time. Use for logging exercises, monitoring metrics, and summarizing performance.",
    "system_prompt": """You are a fitness tracking assistant with expertise in:
- Logging workouts, steps, calories, and activity data
- Tracking progress over time (strength, endurance, consistency)
- Identifying trends and plateaus
- Providing clear summaries and simple insights

Always focus on accurate record-keeping.
Present progress in an easy-to-understand way.
Highlight improvements and areas that need attention.""",
    "tools": [],  
    "model": "openai:gpt-4o-mini",
}


habit_coach = {
    "name": "habit-coach",
    "description": "Coach focused on building and maintaining healthy habits through behavior change techniques and accountability.",
    "system_prompt": """You are a habit coach with expertise in:
- Habit formation and behavior change psychology
- Goal setting and breaking goals into small actions
- Accountability and consistency strategies
- Identifying obstacles and creating practical solutions

Always encourage small, sustainable steps.
Help users design clear daily or weekly routines.
Focus on consistency over perfection.
Provide motivational but practical guidance.""",
    "tools": [], 
    "model": "openai:gpt-4o-mini",
}

# Step 2: Create any additional tools you need


# Step 3: Build the main coordinator agent
# Create the Wellness Coach coordinator
wellness_coach_coordinator = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  
    subagents=[fitness_tracker, habit_coach],
    system_prompt="""You are a Personal Wellness Coach that runs a 30-Day Wellness Challenge.

## Your Role
- Understand each user's unique goals, constraints, and preferences (use get_user_profile first)
- Create a personalized 30-day wellness plan with todos
- Coordinate fitness tracking and habit coaching via subagents
- Track daily progress and adapt recommendations using memory (save_user_preference)
- Store daily check-in notes and weekly summary reports to files

## Workflow
1. **Initial Assessment**: Get user profile; understand goals and any restrictions
2. **Planning**: Create a todo list for the 30-day challenge (plan, check-ins, weekly summaries)
3. **Delegation**: Use subagents for domain-specific content:
   - fitness-tracker: Workouts, activity logging, progress tracking, and fitness summaries
   - habit-coach: Habit formation, routines, accountability, and overcoming obstacles
4. **Daily check-ins**: Store each day's check-in notes in files (e.g. checkins/ or daily/)
5. **Weekly summary**: Save a weekly summary report to files (e.g. reports/week_1_summary.md)
6. **Adaptation**: Use save_user_preference when the user gives feedback so you adapt over time

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress, check-ins, and weekly reports to files so users can reference later""", 
checkpointer=checkpointer
)

In [75]:
# Step 4: Test with a user creating their 30-day challenge
TODO_STORE.clear()

result = wellness_coach_coordinator.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_karla. I'd like to start your 30-Day Wellness Challenge.

I want to:
1. Build a consistent exercise habit (I can do 20–30 min most days)
2. Improve my sleep and stress with better routines
3. Track daily progress and get a weekly summary

Please create my personalized 30-day plan and tell me where you'll save check-ins and weekly reports."""
    }]
}, config={"configurable": {"thread_id": "user_karla_thread"}})

print("Coordinator response (30-day challenge):")
print(result["messages"][-1].content)


Coordinator response (30-day challenge):
Perfect! 🎉 **Your 30-Day Wellness Challenge is now set up!**

## Here's what I've created for you:

### 📋 **Your Personalized Plans**
- **Exercise Plan**: Saved to `/karla_wellness_challenge/plans/30_day_exercise_plan.txt`
- **Sleep & Stress Plan**: Saved to `/karla_wellness_challenge/plans/sleep_stress_management_plan.txt`

### 📱 **Daily Check-ins** will be saved to:
`/karla_wellness_challenge/daily_checkins/`
- Format: `day_01_checkin.md`, `day_02_checkin.md`, etc.

### 📊 **Weekly Summary Reports** will be saved to:  
`/karla_wellness_challenge/weekly_reports/`
- Format: `week_1_summary.md`, `week_2_summary.md`, etc.

### 🎯 **Your Challenge Overview**
I've created a complete overview at `/karla_wellness_challenge/your_30_day_challenge_overview.md`

## What's Next?
1. **Review your plans** - Check out both the exercise and sleep/stress plans
2. **Start tomorrow** - Begin with Day 1 activities  
3. **Daily check-ins** - Message me each day to tr

In [76]:
TODO_STORE.clear()
# Step 5: Simulate a daily check-in and adaptation
check_in_msg = {
    "role": "user",
    "content": """It's day 3. My user_id is user_karla. Here's my check-in:
- Did 25 min walk yesterday, felt good.
- Sleep was still rough (only 5.5 hours). I'm stressed about work.
- I'd like to focus more on wind-down routines.

Please log this check-in, save it to a file, and give me one small adaptation for the next few days."""
}
check_in_result = wellness_coach_coordinator.invoke({
    "messages": check_in_msg
}, config={"configurable": {"thread_id": "user_karla_thread"}})

print("\n" + "=" * 60)
print("Coordinator response (daily check-in & adaptation):")
print(check_in_result["messages"][-1].content)


Coordinator response (daily check-in & adaptation):
## ✅ Day 3 Check-in Logged! 

Great work staying consistent with your exercise - that 25-minute walk is exactly what you need to build the habit! 🚶‍♀️

## 🌙 **Your Small Adaptation for Days 4-6:**

### **"The 9:30 PM Work-to-Rest Transition"** (10-15 minutes)

**Set an alarm for 9:30 PM each night and follow these 4 simple steps:**

1. **Digital Shutdown** (1 min): Turn off all devices - phone, laptop, everything
2. **Work Stress Release** (3 min): Sit quietly and mentally "set aside" work thoughts - maybe jot them in a notebook to deal with tomorrow
3. **4-4-4 Breathing** (5 min): Breathe in for 4, hold for 4, breathe out for 4 - repeat this cycle
4. **Gentle Neck/Shoulder Stretches** (5 min): Release the physical tension from work stress

**Why this works:** It creates a clear boundary between "work stress" and "sleep time," giving your nervous system permission to start winding down.

## 📊 **Your Progress Summary:**
- ✅ **Exercise

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)